## Model Training on Custom Mnist Dataset 

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [3]:
dataset_path = "dataset"

In [5]:
def preprocess_image(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize to [0, 1]
    return image, label

# Load the training dataset
train_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,  # 20% of data for validation
    subset="training",
    seed=123,
    image_size=(28, 28),
    batch_size=32,
    color_mode='rgb'  # Load images as RGB
)

# Load the validation dataset
validation_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,  # 20% of data for validation
    subset="validation",
    seed=123,
    image_size=(28, 28),
    batch_size=32,
    color_mode='rgb'  # Load images as RGB
)

Found 3558 files belonging to 10 classes.
Using 2847 files for training.
Found 3558 files belonging to 10 classes.
Using 711 files for validation.


In [6]:
# Apply the preprocessing function to convert to grayscale and normalize
train_dataset = train_dataset.map(preprocess_image)
validation_dataset = validation_dataset.map(preprocess_image)

In [7]:
for images, labels in train_dataset.take(1):
    print(images.shape)  # Should be (batch_size, 28, 28, 1) after conversion to grayscale
    print(tf.reduce_min(images), tf.reduce_max(images))  # Should be between 0 and 1


(32, 28, 28, 1)
tf.Tensor(5.0254965, shape=(), dtype=float32) tf.Tensor(199.89449, shape=(), dtype=float32)


In [12]:
from tensorflow.keras import layers, models

# Create a simple sequential model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # Assuming 10 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [13]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20  # Adjust the number of epochs as needed
)


Epoch 1/20
89/89 [==============================] - 3s 24ms/step - loss: 5.3845 - accuracy: 0.1992 - val_loss: 1.8898 - val_accuracy: 0.3235
Epoch 2/20
89/89 [==============================] - 2s 23ms/step - loss: 1.4341 - accuracy: 0.4991 - val_loss: 1.2190 - val_accuracy: 0.5513
Epoch 3/20
89/89 [==============================] - 2s 23ms/step - loss: 0.9025 - accuracy: 0.7004 - val_loss: 0.8198 - val_accuracy: 0.7229
Epoch 4/20
89/89 [==============================] - 2s 24ms/step - loss: 0.5566 - accuracy: 0.8166 - val_loss: 0.6558 - val_accuracy: 0.7764
Epoch 5/20
89/89 [==============================] - 2s 23ms/step - loss: 0.3232 - accuracy: 0.9017 - val_loss: 0.4238 - val_accuracy: 0.8579
Epoch 6/20
89/89 [==============================] - 2s 24ms/step - loss: 0.1896 - accuracy: 0.9449 - val_loss: 0.3595 - val_accuracy: 0.8805
Epoch 7/20
89/89 [==============================] - 2s 24ms/step - loss: 0.1171 - accuracy: 0.9656 - val_loss: 0.3895 - val_accuracy: 0.8847
Epoch 8/20
89

In [14]:
model.save('custom_mnist_model.h5')

c:\Users\ncai\anaconda3\envs\yolo_gpu\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Model Training on Mnist dataset

In [15]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and reshape the data
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Create the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # Assuming 10 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(train_images, train_labels, epochs=20, batch_size=32, validation_data=(test_images, test_labels))


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [16]:
model.save('mnist_model.h5')

## Cross Validdation of Custom Model on the Mnist testing dataset and Vice Versa

In [17]:
from tensorflow.keras.models import load_model

# Load the custom MNIST model
custom_mnist_model = load_model('custom_mnist_model.h5')

# Load the MNIST model
mnist_model = load_model('mnist_model.h5')


In [18]:
loss, accuracy = custom_mnist_model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 2.3729 - accuracy: 0.1114


In [19]:
loss, accuracy = mnist_model.evaluate(validation_dataset)

23/23 [==============================] - 0s 10ms/step - loss: 1132.9048 - accuracy: 0.1125
